# Большая домашка 1: сегментация 

Товарищи, мы все уже великие специалисты в нейронных сетях, поэтому пришло время показать это миру. Мы будем решать конкурс от роснефти по сегментации изображений. У нас будут изображения с трехмерного куба, где надо найти различные области на этих снимках. А дальше уже смотрите сами :)

Более подробно о данном конкурсе можно посмотреть [вот тута](https://boosters.pro/championship/seismic_challenge/overview)

В рамках данной домашки попробуем вспомнить все, что нам мы делали. Дальше будет о подгрузке данных и небольшие направления деятельности, но в основном ваше фантазия решает этот конкурс :)

In [1]:
import tensorflow as tf
# import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import cv2 # подгрузка и работа с данными
import os
import tqdm # счетчик и красивый прогресс бар
from sklearn.model_selection import train_test_split  

In [2]:
IMAGE_PATH = 'data/train/images/'
MASK_PATH = 'data/train/train.csv'
# TEST_PATH = 'data/test/images/'
# SAMPLE_PATH = 'make_rofneft_predict/data/sample_submission.csv'

def rle2mask(mask_rle, shape):
    # эта функция взята с конкурса и я её оставил как есть
    """
    mask_rle: run-length as string formated (start length)ы
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background
    """
    if mask_rle != mask_rle:
        return np.zeros_like(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [3]:
class TrainDataset():
    # немного переписанный загрузчик данных опять же с конкурса
    def __init__(self, image_dir, train):
        self.image_dir = image_dir
        self.train = train

    def __getitem__(self, img_name):
#         img_name = self.train.iloc[index]['ImageId']
        path = os.path.join(self.image_dir, img_name)
        img = cv2.imread(path)
        img = cv2.resize(img,(384,384))

        ce_mask = [
            (i + 1) * rle2mask(rle, shape=(img.shape[0], img.shape[1])) \
                for i, rle in enumerate(self.train[self.train['ImageId']==img_name]['EncodedPixels'])
        ]
        ce_mask = np.sum(ce_mask, axis=0, dtype=np.float32)


        result = {
            'image':img, 
            'mask': ce_mask,
            'pick_name':img_name
        }
        return result
    
    def __len__(self, ):
        return len(self.train)
    
    
train = pd.read_csv(MASK_PATH)

train_dataset = TrainDataset(
    image_dir=IMAGE_PATH, 
    train=train)
    
class DataGenerator_manual(tf.keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self,train_dataset,train_indexes,batch_size):
        self.train_dataset = train_dataset
        self.train_indexes = train_indexes
        self.batch_size = batch_size
         
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.train_indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        self.index = index
        index_sample = np.random.choice(self.train_indexes,self.batch_size)
        X_full = []
        y_full = []
        for img_name in index_sample:
            x_step,y_step = self.getitem_step(img_name)
            X_full.append(x_step)
            y_full.append(y_step)
        return  tf.convert_to_tensor(X_full), tf.convert_to_tensor(y_full)

    def getitem_step(self, img_name):
        slice_step = train_dataset[img_name]
        
        x_images = np.array(slice_step['image'][:,:,0])
        x_images_norm = x_images/255

        return tf.reshape(tf.convert_to_tensor(x_images_norm),(384,384,1)), \
                tf.convert_to_tensor(slice_step['mask'])


In [4]:
train_indexes,test_indexes = train_test_split(train['ImageId'],test_size=0.3)
my_generator_try = DataGenerator_manual(train_dataset,train_indexes,batch_size=32)

In [6]:
model_trunc.fit(my_generator_try,steps_per_epoch=100,epochs=10,verbose=1)


  ...
    to  
  ['...']
Train for 100 steps
Epoch 1/10
100/100 [==============================] - 73s 730ms/step - loss: 1.6811 - sparse_categorical_accuracy: 0.4599
Epoch 2/10
100/100 [==============================] - 71s 709ms/step - loss: 0.9430 - sparse_categorical_accuracy: 0.7185
Epoch 3/10
100/100 [==============================] - 71s 708ms/step - loss: 0.5834 - sparse_categorical_accuracy: 0.8580
Epoch 4/10
100/100 [==============================] - 71s 708ms/step - loss: 0.3687 - sparse_categorical_accuracy: 0.9210
Epoch 5/10
100/100 [==============================] - 71s 708ms/step - loss: 0.2654 - sparse_categorical_accuracy: 0.9363
Epoch 6/10
 94/100 [===========================>..] - ETA: 4s - loss: 0.2166 - sparse_categorical_accuracy: 0.9420

KeyboardInterrupt: 

Кстати, посмотрев на данные, какие особенности данных мы видим?

In [ ]:
from tqdm import tqdm

In [ ]:
x_images = []
y_images = []
name_images = []
for i in tqdm(range(300)):
    slice_step = train_dataset[i]
    if 'inline' in slice_step['pick_name']:
        pickk_name = 'inline'
    else:
        pickk_name = 'xline'
    name_images.append(pickk_name)
    x_images.append(slice_step['image'][:,:,0])
    y_images.append(slice_step['mask']) 

In [ ]:
x_images = np.array(x_images)
x_images_norm = x_images/255
y_images = np.array(y_images,dtype=np.int)

In [ ]:
plt.imshow(x_images[0],cmap='gray')

In [ ]:
plt.imshow(y_images[0])



In [ ]:
plt.imshow(y_images[3507])

In [ ]:
name_images.index('xline')

Такс, что мы хотим делать с данными дальше ? Можем ли мы что-то предположить заранее?

In [ ]:
#### Ваш код игры с данными

В самой задачке на бустере используется метрика dice по границе и всей области.
Мы упростим задачу, просто берем dice по области.

Собственно нам надо реализовать следующую формулу

\begin{equation}
dice = \sum_{k=0}^7(\cfrac{2*|prediction \cap truch|}{|prediction| + |truch|})
\end{equation}

Ну и чтобы в случае чего не делить на ноль, надо добавить небольшой эпсилон к верхней и нижней части.

In [ ]:
def dice_metric(y_true, y_pred):
    smooth = 0.00001
    dice_list_value = tf.constant(0.) 
    for val in range(0,8):
        y_true_step = tf.cast((y_true==val),'float32')
        y_pred_step = tf.cast((y_pred==val),'float32')
        intersection = tf.reduce_sum(y_true_step*y_pred_step,axis=(1,2))
        union = tf.reduce_sum(y_true_step,axis=(1,2))+tf.reduce_sum(y_pred_step,axis=(1,2))
        dice = (2*intersection+smooth)/union
        dice_list_value += dice
    return tf.reduce_mean(dice_list_value/8)


Придумываем бейзлайн для всего нашего кусочка - должны же мы его превзойти!

In [ ]:
# Код бейзлайна
# Усреднить маски и сравнивать с полученной маской


 #### Ваш код игры с данными

classes=[0,1,2,3,4,5,6,7]
dict_start={}
dict_end={}
for ob_class in classes:
    dict_start[ob_class]=[]
    dict_end[ob_class]=[]
for y_trr in y_images:
    first_col=y_trr[:,0]
    dict_start[0].append(0)
    for ob_class in classes:
        for i in range(1,len(first_col)):
            if first_col[i]==ob_class and first_col[i-1]!=ob_class:   
                dict_start[ob_class].append(i)
            elif first_col[i-1]==ob_class and first_col[i]!=ob_class:
                dict_end[ob_class].append(i)
                break

In [ ]:
dict_total={}
for ob_class in classes:
    dict_total[ob_class]=[np.median(dict_start[ob_class]).astype(int),np.median(dict_end[ob_class]).astype(int)]



In [ ]:
y_images_baseline=np.zeros(y_images[0].shape,dtype=int)
for ob_cl,bound in dict_total.items():
    y_images_baseline[bound[0]:bound[1],:]=ob_cl
plt.figure(figsize=(20,10))
plt.imshow(y_images_baseline)
plt.title("y_images_baseline")



In [ ]:
y_images_baseline=tf.convert_to_tensor([y_images_baseline]*10, dtype=tf.float32)

In [ ]:
dice_metric(y_images[0:10],y_images_baseline)

Ну что же, теперь пришло время к моделям!
Начнем с простой модельки - fully-convolution-net.

И да - архитектура на ваш вкус и цвет :)

Но прежде чем начинать, какие callback нам точно потребуются? (вне зависимости от архитектуры)

In [ ]:
# Собираем fully-convolution-net

Собираем u-net. Смотрим как оно :)

In [ ]:
# Собираем u-net

Ну чтож, время вашего творчества - меняем архитектуры, смотрим и вспоминаем, что же ещё мы не учли и что может нам помочь. Да и вообще -какие подходы ещё не использованы! Ещё имеет смысл попробовать transfer learning!

In [ ]:
### играемся на все деньги

И доп задачка для всех - а что если обучить часть (или весь) unet как автоэнкодер...


In [ ]:
##############################